In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import tweepy
import re
import json
import os 

In [7]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [8]:
covid_tweets_clean= pd.read_json('covid_tweets_clean.json')

In [4]:
covid_tweets_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001862 entries, 0 to 10001861
Data columns (total 2 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   text      object
 1   location  object
dtypes: object(2)
memory usage: 152.6+ MB


In [9]:
#number of unique locations 
covid_tweets_clean['location'].nunique()

512706

In [ ]:
#top 40 locations 
covid_tweets_clean['location'].value_counts().head(40)

In [12]:
scores = []
# Declare variables for scores
compound_list = []
positive_list = []
negative_list = []
neutral_list = []
for i in range(covid_tweets_clean['text'].shape[0]):
#print(analyser.polarity_scores(sentiments_pd['text'][i]))
    compound = analyzer.polarity_scores(covid_tweets_clean['text'][i])["compound"]
    pos = analyzer.polarity_scores(covid_tweets_clean['text'][i])["pos"]
    neu = analyzer.polarity_scores(covid_tweets_clean['text'][i])["neu"]
    neg = analyzer.polarity_scores(covid_tweets_clean['text'][i])["neg"]
    
    scores.append({"Compound": compound,
                       "Positive": pos,
                       "Negative": neg,
                       "Neutral": neu
                  })

In [14]:
sentiments_score = pd.DataFrame.from_dict(scores)

In [15]:
covid_tweets_sentiment = covid_tweets_clean.join(sentiments_score)
covid_tweets_sentiment.head()

,text,location,Compound,Positive,Negative,Neutral
0,They said ain\\u2019t no way covid still 19 t...,New York,-0.2960,0.000,0.145,0.855
1,I\\u2019ve just tested positive for Covid wha...,Slidell,0.6072,0.265,0.000,0.735
2,The coronavirus has messed up even more and al...,null,-0.6548,0.000,0.349,0.651
3,How he had a load of conspiracy nuts on string...,Bristol,-0.6908,0.000,0.305,0.695
4,But this might work and if it does we can el...,Gold Coast,-0.8720,0.000,0.357,0.643


In [17]:
covid_tweets_sentiment['Compound_sentiment'] = covid_tweets_sentiment['Compound'].apply(lambda c: 'pos' if c >=0 else 'neg')

covid_tweets_sentiment.head()

,text,location,Compound,Positive,Negative,Neutral,Compound_sentiment
0,They said ain\\u2019t no way covid still 19 t...,New York,-0.2960,0.000,0.145,0.855,neg
1,I\\u2019ve just tested positive for Covid wha...,Slidell,0.6072,0.265,0.000,0.735,pos
2,The coronavirus has messed up even more and al...,null,-0.6548,0.000,0.349,0.651,neg
3,How he had a load of conspiracy nuts on string...,Bristol,-0.6908,0.000,0.305,0.695,neg
4,But this might work and if it does we can el...,Gold Coast,-0.8720,0.000,0.357,0.643,neg


In [18]:
#save the file into one json file 
covid_tweets_sentiment.to_json('./covid_tweets_sentiment.json', orient='records')